In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import sys
import json
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm
import numpy as np
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

path = '/content/drive/MyDrive/magistrale/nlp/nlp2023-hw1-main/data/'
sys.path.append(path)

/home/ghost/Desktop/nlp_env/nlp_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!ls -lat

--2023-04-23 14:59:31--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-04-23 14:59:31--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-23 14:59:32--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [3]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.id_label = {}
        self.idx = 0


        #forse non devo usare 0,...,10 ma numeri mai usati da self.x
        self.label2idx = {
         'O': 0,
         'B-ACTION':1,
         'B-CHANGE':2,
         'B-SCENARIO':3,
         'B-POSSESSION':4,
         'B-SENTIMENT':5,
         'I-SCENARIO':6,
         'I-ACTION':7,
         'I-CHANGE':8,
         'I-SENTIMENT':9,
         'I-POSSESSION':10
          }

    def add_word(self, word):
        word = word.lower()
        if word not in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
            


    def __len__(self):
        return self.idx #forse devo fare -11


In [7]:
class TextProcessTrain(object):
    # We indicate 1 as the unknown word
    # And we indicate 0 as the padding

    def __init__(self,dictionary):
        self.dictionary = dictionary 
        self.x = []
        self.y = []
        self.max_length_word = 0
        self.len = 0
        # data loading


    def get_data_train(self, datapath):
        with open(path + datapath, 'r') as f:
          for line in f:
            data = json.loads(line)
            self.x.append(data['tokens'])
            self.y.append(data['labels'])

 
          for tokens in self.x:
            for token in tokens:
              self.dictionary.add_word(token)
              
        self.max_length_word = max(len(t[0]) for t in self.x)
        #maybe is mnot necessary to add padding and unk to dicitonary
        self.dictionary.add_word('<pad>')
        self.dictionary.add_word('<unk>')
  
        self.n_tokens = len(self.dictionary)

    def get_data_test(self, datapath, max_length_word):
      # here the vocabulary is already defined

        with open(path + datapath, 'r') as f:
          for line in f:
            data = json.loads(line)
            self.x.append(data['tokens'])
            self.y.append(data['labels'])

        self.max_length_word = max_length_word


    def __getitem__(self, index):
      # so if the word doesn't exists ill return 1
      #word = torch.LongTensor([ self.dictionary.word2idx.get(word, 1) or 1 for word in self.x[index] ])
      try:  

        self.x[index] 
      except:

        print(index)
        print(len(self.x))

      word = torch.LongTensor([ self.dictionary.word2idx.get(word, self.dictionary.word2idx["<unk>"]) for word in self.x[index] ])
      label = torch.LongTensor([ self.dictionary.label2idx[label] for label in self.y[index] ])

      # add padding to all the words and label so that they all are the same length
      word = torch.nn.functional.pad(word, (0, self.max_length_word - len(word)), "constant",self.dictionary.word2idx["<pad>"]  )
      label = torch.nn.functional.pad(label, (0, self.max_length_word - len(label)), "constant",self.dictionary.word2idx["<pad>"]  )

      return  word,label

        #return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)
        

In [8]:
class TextGenerator(nn.Module):
    
    def __init__(self, output_layer_size, vocab_size, embed_size, hidden_size, num_layers, batch_size,dictionary):
        super(TextGenerator, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.output_layer_size = output_layer_size
        self.num_layers = num_layers
        self.hidden = self.init_hidden()

        self.embed = nn.Embedding(vocab_size, embed_size, padding_idx=dictionary.word2idx["<pad>"])
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear_01 = nn.Linear(hidden_size, output_layer_size)
       # self.linear_02 = nn.Linear(2048, 1024)
        #self.linear_03 = nn.Linear(1024, 512)
        #self.linear_04 = nn.Linear(512, output_layer_size)


    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_size),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_size))
        
    def forward(self, x):
        # Perform Word Embedding 
        embeds = self.embed(x)
        #print(embeds.shape)
        #Reshape the input tensor
        #x = x.view(batch_size,timesteps,embed_size)
        #print(self.hidden[0].shape)
        #packed_embeds = pack_padded_sequence(a, [6], batch_first=True, enforce_sorted=False)
        out, self.hidden = self.lstm(embeds, self.hidden)
        # Flatten output tensor for use in the linear layer
        #print(out.shape)
        out = self.linear_01(out)
        #out = F.relu(self.linear_01(out[:, -1]))
        #out = self.linear_02(out)
        #out = self.linear_01(out[:, -1])
        #out = self.linear_02(out)
        #out = self.linear_03(out)
        #out = self.linear_04(out)
        return out

In [9]:
embed_size = 32    #Input features to the LSTM
hidden_size = 512  #Number of LSTM units
num_layers = 1
num_epochs = 20
batch_size = 16
learning_rate = 0.18


In [10]:
dictionary = Dictionary()
corpusTrain = TextProcessTrain(dictionary)
corpusTrain.get_data_train("train.jsonl")

corpusTest = TextProcessTrain(dictionary)
corpusTest.get_data_test("test.jsonl",corpusTrain.max_length_word)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/magistrale/nlp/nlp2023-hw1-main/data/train.jsonl'

In [ ]:
#train_loader = DataLoader(dataset=corpus, batch_size=20,shuffle=False, collate_fn=lambda x: x )
print(batch_size)
train_loader = DataLoader(dataset=corpusTrain, batch_size=batch_size,shuffle=False,pin_memory=True)
test_loader = DataLoader(dataset=corpusTest, batch_size=batch_size,shuffle=False,pin_memory=True,drop_last=True,num_workers=0)
# Display image and label.
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

for step, sample in tqdm(enumerate( train_loader ),desc="Batch"):
    print(sample[0].shape)
    break

16
Feature batch shape: torch.Size([16, 20])
Labels batch shape: torch.Size([16, 20])


Batch: 0it [00:00, ?it/s]

torch.Size([16, 20])


In [ ]:
vocab_size = len(corpusTrain.dictionary)
output_layer_size = len(dictionary.label2idx)
print(vocab_size)
print(output_layer_size)

33081
11


In [ ]:
model = TextGenerator(output_layer_size, vocab_size, embed_size, hidden_size, num_layers,batch_size,dictionary)
print(model)

TextGenerator(
  (embed): Embedding(33081, 32, padding_idx=33079)
  (lstm): LSTM(32, 512, batch_first=True)
  (linear_01): Linear(in_features=512, out_features=11, bias=True)
)


In [ ]:
loss_fn = nn.CrossEntropyLoss(ignore_index=dictionary.word2idx["<pad>"])
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
print(dictionary.word2idx["<pad>"])

33079


In [ ]:
train_loss = 0.0
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size),
              torch.zeros(num_layers, batch_size, hidden_size))

    epoch_loss = 0.0
    len_train = 0
    with tqdm(enumerate( train_loader ),desc="Batch", leave=False) as data:
      for step, sample in data:
          model.zero_grad()
          inputs = sample[0]
          targets = sample[1] 
          model.hidden = model.init_hidden()
          # Also, we need to clear out the hidden state of the LSTM, detaching it from its
          # history on the last instance.           targets.reshape(-1)
          

          outputs = model(inputs)

          #print(targets[:,:11].shape)
          loss = loss_fn(outputs.view(-1, outputs.shape[-1]), targets.view(-1))

         
          loss.backward()
          #Perform Gradient Clipping. clip_value (float or int) is the maximum allowed value of the gradients 
          #The gradients are clipped in the range [-clip_value, clip_value]. This is to prevent the exploding gradient problem
          clip_grad_norm(model.parameters(), 0.5)
          optimizer.step()
          epoch_loss += loss.item()
          len_train += 1
    
          avg_epoch_loss = epoch_loss / len_train     
          if step % 10 == 0:
            print('Epoch: {} avg loss = {:0.4f} loss = {:0.4f} '.format(epoch, avg_epoch_loss,loss.item()))

          train_loss += avg_epoch_loss

avg_epoch_loss = train_loss / num_epochs

print("avarage epoch loss = " + str(avg_epoch_loss))

Batch: 0it [00:00, ?it/s]

<ipython-input-96-1f93d9e8ad32>:28: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  clip_grad_norm(model.parameters(), 0.5)


Epoch: 0 avg loss = 2.3768 loss = 2.3768 
Epoch: 0 avg loss = 1.9240 loss = 1.4777 
Epoch: 0 avg loss = 1.4854 loss = 0.7631 
Epoch: 0 avg loss = 1.1999 loss = 0.6433 
Epoch: 0 avg loss = 1.0389 loss = 0.5358 
Epoch: 0 avg loss = 0.9369 loss = 0.5178 
Epoch: 0 avg loss = 0.8718 loss = 0.5834 
Epoch: 0 avg loss = 0.8246 loss = 0.5668 
Epoch: 0 avg loss = 0.7797 loss = 0.4380 
Epoch: 0 avg loss = 0.7480 loss = 0.4998 
Epoch: 0 avg loss = 0.7218 loss = 0.4876 
Epoch: 0 avg loss = 0.7008 loss = 0.5830 
Epoch: 0 avg loss = 0.6868 loss = 0.4083 
Epoch: 0 avg loss = 0.6703 loss = 0.4963 
Epoch: 0 avg loss = 0.6570 loss = 0.4956 
Epoch: 0 avg loss = 0.6453 loss = 0.4991 
Epoch: 0 avg loss = 0.6329 loss = 0.4750 
Epoch: 0 avg loss = 0.6232 loss = 0.4140 
Epoch: 0 avg loss = 0.6160 loss = 0.5267 
Epoch: 0 avg loss = 0.6068 loss = 0.4126 
Epoch: 0 avg loss = 0.6019 loss = 0.5123 
Epoch: 0 avg loss = 0.5966 loss = 0.6201 
Epoch: 0 avg loss = 0.5905 loss = 0.3945 
Epoch: 0 avg loss = 0.5851 loss = 

Batch: 0it [00:00, ?it/s]

Epoch: 1 avg loss = 0.4418 loss = 0.4418 
Epoch: 1 avg loss = 0.4526 loss = 0.4300 
Epoch: 1 avg loss = 0.4445 loss = 0.5891 
Epoch: 1 avg loss = 0.4453 loss = 0.5353 
Epoch: 1 avg loss = 0.4423 loss = 0.4250 
Epoch: 1 avg loss = 0.4401 loss = 0.4317 
Epoch: 1 avg loss = 0.4431 loss = 0.5038 
Epoch: 1 avg loss = 0.4445 loss = 0.4865 
Epoch: 1 avg loss = 0.4384 loss = 0.3593 
Epoch: 1 avg loss = 0.4378 loss = 0.4298 
Epoch: 1 avg loss = 0.4362 loss = 0.4389 
Epoch: 1 avg loss = 0.4359 loss = 0.5282 
Epoch: 1 avg loss = 0.4382 loss = 0.3722 
Epoch: 1 avg loss = 0.4370 loss = 0.4502 
Epoch: 1 avg loss = 0.4369 loss = 0.4683 
Epoch: 1 avg loss = 0.4364 loss = 0.4364 
Epoch: 1 avg loss = 0.4345 loss = 0.4288 
Epoch: 1 avg loss = 0.4338 loss = 0.3610 
Epoch: 1 avg loss = 0.4342 loss = 0.4815 
Epoch: 1 avg loss = 0.4321 loss = 0.3481 
Epoch: 1 avg loss = 0.4337 loss = 0.4422 
Epoch: 1 avg loss = 0.4344 loss = 0.5595 
Epoch: 1 avg loss = 0.4341 loss = 0.3599 
Epoch: 1 avg loss = 0.4339 loss = 

...

Epoch: 17 avg loss = 0.2251 loss = 0.2251 
Epoch: 17 avg loss = 0.2208 loss = 0.1734 
Epoch: 17 avg loss = 0.2223 loss = 0.3269 
Epoch: 17 avg loss = 0.2243 loss = 0.3124 
Epoch: 17 avg loss = 0.2229 loss = 0.2059 
Epoch: 17 avg loss = 0.2217 loss = 0.2417 
Epoch: 17 avg loss = 0.2221 loss = 0.2967 
Epoch: 17 avg loss = 0.2235 loss = 0.3162 
Epoch: 17 avg loss = 0.2212 loss = 0.2055 
Epoch: 17 avg loss = 0.2221 loss = 0.1808 
Epoch: 17 avg loss = 0.2193 loss = 0.1908 
Epoch: 17 avg loss = 0.2196 loss = 0.2329 
Epoch: 17 avg loss = 0.2188 loss = 0.1899 
Epoch: 17 avg loss = 0.2188 loss = 0.2400 
Epoch: 17 avg loss = 0.2185 loss = 0.2045 
Epoch: 17 avg loss = 0.2186 loss = 0.2509 
Epoch: 17 avg loss = 0.2176 loss = 0.2019 
Epoch: 17 avg loss = 0.2189 loss = 0.1868 
Epoch: 17 avg loss = 0.2179 loss = 0.2739 
Epoch: 17 avg loss = 0.2167 loss = 0.1390 
Epoch: 17 avg loss = 0.2182 loss = 0.1872 
Epoch: 17 avg loss = 0.2182 loss = 0.2314 
Epoch: 17 avg loss = 0.2185 loss = 0.2054 
Epoch: 17 a

Batch: 0it [00:00, ?it/s]

Epoch: 18 avg loss = 0.2207 loss = 0.2207 
Epoch: 18 avg loss = 0.2154 loss = 0.1687 
Epoch: 18 avg loss = 0.2170 loss = 0.3224 
Epoch: 18 avg loss = 0.2189 loss = 0.3046 
Epoch: 18 avg loss = 0.2176 loss = 0.1993 
Epoch: 18 avg loss = 0.2165 loss = 0.2363 
Epoch: 18 avg loss = 0.2168 loss = 0.2911 
Epoch: 18 avg loss = 0.2182 loss = 0.3096 
Epoch: 18 avg loss = 0.2160 loss = 0.2023 
Epoch: 18 avg loss = 0.2169 loss = 0.1768 
Epoch: 18 avg loss = 0.2142 loss = 0.1857 
Epoch: 18 avg loss = 0.2145 loss = 0.2269 
Epoch: 18 avg loss = 0.2137 loss = 0.1833 
Epoch: 18 avg loss = 0.2136 loss = 0.2348 
Epoch: 18 avg loss = 0.2134 loss = 0.1989 
Epoch: 18 avg loss = 0.2134 loss = 0.2463 
Epoch: 18 avg loss = 0.2125 loss = 0.1948 
Epoch: 18 avg loss = 0.2138 loss = 0.1864 
Epoch: 18 avg loss = 0.2128 loss = 0.2696 
Epoch: 18 avg loss = 0.2117 loss = 0.1344 
Epoch: 18 avg loss = 0.2132 loss = 0.1842 
Epoch: 18 avg loss = 0.2131 loss = 0.2244 
Epoch: 18 avg loss = 0.2135 loss = 0.2024 
Epoch: 18 a

Batch: 0it [00:00, ?it/s]

Epoch: 19 avg loss = 0.2165 loss = 0.2165 
Epoch: 19 avg loss = 0.2103 loss = 0.1646 
Epoch: 19 avg loss = 0.2120 loss = 0.3181 
Epoch: 19 avg loss = 0.2137 loss = 0.2974 
Epoch: 19 avg loss = 0.2125 loss = 0.1931 
Epoch: 19 avg loss = 0.2116 loss = 0.2316 
Epoch: 19 avg loss = 0.2119 loss = 0.2858 
Epoch: 19 avg loss = 0.2133 loss = 0.3030 
Epoch: 19 avg loss = 0.2110 loss = 0.1993 
Epoch: 19 avg loss = 0.2121 loss = 0.1733 
Epoch: 19 avg loss = 0.2094 loss = 0.1810 
Epoch: 19 avg loss = 0.2098 loss = 0.2209 
Epoch: 19 avg loss = 0.2089 loss = 0.1772 
Epoch: 19 avg loss = 0.2087 loss = 0.2295 
Epoch: 19 avg loss = 0.2085 loss = 0.1936 
Epoch: 19 avg loss = 0.2086 loss = 0.2419 
Epoch: 19 avg loss = 0.2077 loss = 0.1884 
Epoch: 19 avg loss = 0.2090 loss = 0.1860 
Epoch: 19 avg loss = 0.2080 loss = 0.2652 
Epoch: 19 avg loss = 0.2069 loss = 0.1302 
Epoch: 19 avg loss = 0.2084 loss = 0.1814 
Epoch: 19 avg loss = 0.2084 loss = 0.2178 
Epoch: 19 avg loss = 0.2088 loss = 0.1995 
Epoch: 19 a

In [ ]:
a = torch.tensor([[9,2,3,4,5,0,0,0,0]])
a = torch.nn.functional.pad(a, (0, 3))

print(a)
b = pack_padded_sequence(a, [6], batch_first=True, enforce_sorted=False)
print(b[0])

tensor([[9, 2, 3, 4, 5, 0, 0, 0, 0, 0, 0, 0]])
tensor([9, 2, 3, 4, 5, 0])


In [ ]:
print(len(corpusTest))

2000


In [ ]:
def F_score(logit, label, threshold=0.5, beta=2):
    prob = torch.sigmoid(logit)
    prob = prob > threshold
    label = label > threshold

    TP = (prob & label).sum(1).float()
    TN = ((~prob) & (~label)).sum(1).float()
    FP = (prob & (~label)).sum(1).float()
    FN = ((~prob) & label).sum(1).float()

    precision = TP / (TP + FP + 1e-12)
    recall = TP / (TP + FN + 1e-12)
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2.mean(0)


def label2onehot(label, num_classes=7178):
    onehot = label.new(label.size(0), num_classes)
    onehot.zero_()
    onehot.scatter_(1, label, 1)
    return onehot



torch.set_printoptions(sci_mode=False,threshold=sys.maxsize)
po = 0

f2_score = 0
# Test the model
with torch.no_grad():
    with tqdm(enumerate( test_loader ),desc="Batch", leave=False) as data:
      for step, sample in data:
            inputs = sample[0]
            targets = sample[1]

            #print(targets)
            output = model(inputs)
            #print(torch.argmax(output,-1))

            fscore = F_score(torch.argmax(output,-1), targets)
            #print(fscore)
            f2_score += fscore
 

            po = step

print(po)
print(f2_score/po)




Batch: 0it [00:00, ?it/s]

124
tensor(0.4664)


In [ ]:
glove_tokens = []
glove_embeddings = []

with open('./glove.6B.50d.txt', 'rt') as f:

  full_content = f.read().strip().split('\n')
  for i in range(len(full_content)):
      i_word = full_content[i].split(' ')[0]
      i_embeddings = [float(val) for val in full_content[i].split(' ')[1:]]
      glove_tokens.append(i_word)
      glove_embeddings.append(i_embeddings)


In [ ]:
glove_embeddings = torch.tensor(glove_embeddings)

#(1) the pad token embedding as zeros and (2) the unk token embedding as the mean of all other embeddings.


#add <pad> and <unk> where <pad> is indicated as 0, <unk> as 1
glove_tokens.insert(0,"<unk>")
unk_emb = torch.mean(glove_embeddings, dim=0, keepdim=True)
glove_embeddings = torch.cat((unk_emb, glove_embeddings))

glove_tokens.insert(0,"<pad>")
pad_emb = torch.zeros((1, glove_embeddings.shape[1]))
glove_embeddings = torch.cat((pad_emb, glove_embeddings))

print(glove_tokens[:10])
print(glove_embeddings[1])

<ipython-input-196-4051335dc9b6>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  glove_embeddings = torch.tensor(glove_embeddings)


['<pad>', '<unk>', '<pad>', '<unk>', '<pad>', '<unk>', 'the', ',', '.', 'of']
tensor([-0.1292, -0.2887, -0.0122, -0.0568, -0.2021, -0.0839,  0.3336,  0.1604,
         0.0387,  0.1783,  0.0470, -0.0029,  0.2910,  0.0461, -0.2092, -0.0661,
        -0.0682,  0.0767,  0.3134,  0.1785, -0.1226, -0.0992, -0.0750,  0.0641,
         0.1444,  0.6089,  0.1746,  0.0534, -0.0127,  0.0347, -0.8124, -0.0469,
         0.2019,  0.2031, -0.0394,  0.0697, -0.0155, -0.0341, -0.0653,  0.1225,
         0.1399, -0.1745, -0.0801,  0.0850, -0.0104, -0.1370,  0.2013,  0.1007,
         0.0065,  0.0169])
